<a href="https://colab.research.google.com/github/AI4Finance-Foundation/FinRL/blob/master/FinRL_StockTrading_NeurIPS_2018.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Reinforcement Learning for Stock Trading from Scratch: Multiple Stock Trading

* **Pytorch Version** 



# Content

* [1. Problem Definition](#0)
* [2. Getting Started - Load Python packages](#1)
    * [2.1. Install Packages](#1.1)    
    * [2.2. Check Additional Packages](#1.2)
    * [2.3. Import Packages](#1.3)
    * [2.4. Create Folders](#1.4)
* [3. Download Data](#2)
* [4. Preprocess Data](#3)        
    * [4.1. Technical Indicators](#3.1)
    * [4.2. Perform Feature Engineering](#3.2)
* [5.Build Environment](#4)  
    * [5.1. Training & Trade Data Split](#4.1)
    * [5.2. User-defined Environment](#4.2)   
    * [5.3. Initialize Environment](#4.3)    
* [6.Implement DRL Algorithms](#5)  
* [7.Backtesting Performance](#6)  
    * [7.1. BackTestStats](#6.1)
    * [7.2. BackTestPlot](#6.2)   
    * [7.3. Baseline Stats](#6.3)   
    * [7.3. Compare to Stock Market Index](#6.4)   
* [RLlib Section](#7)            

<a id='0'></a>
# Part 1. Problem Definition

This problem is to design an automated trading solution for single stock trading. We model the stock trading process as a Markov Decision Process (MDP). We then formulate our trading goal as a maximization problem.

The algorithm is trained using Deep Reinforcement Learning (DRL) algorithms and the components of the reinforcement learning environment are:


* Action: The action space describes the allowed actions that the agent interacts with the
environment. Normally, a ∈ A includes three actions: a ∈ {−1, 0, 1}, where −1, 0, 1 represent
selling, holding, and buying one stock. Also, an action can be carried upon multiple shares. We use
an action space {−k, ..., −1, 0, 1, ..., k}, where k denotes the number of shares. For example, "Buy
10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or −10, respectively

* Reward function: r(s, a, s′) is the incentive mechanism for an agent to learn a better action. The change of the portfolio value when action a is taken at state s and arriving at new state s',  i.e., r(s, a, s′) = v′ − v, where v′ and v represent the portfolio
values at state s′ and s, respectively

* State: The state space describes the observations that the agent receives from the environment. Just as a human trader needs to analyze various information before executing a trade, so
our trading agent observes many different features to better learn in an interactive environment.

* Environment: Dow 30 consituents


The data of the single stock that we will be using for this case study is obtained from Yahoo Finance API. The data contains Open-High-Low-Close price and volume.


<a id='1'></a>
# Part 2. Getting Started- Load Python Packages

<a id='1.1'></a>
## 2.1. Install all the packages through FinRL library


In [ ]:
## install finrl library
!pip install git+https://github.com/AI4Finance-Foundation/FinRL-Library.git

  Cloning https://github.com/AI4Finance-Foundation/FinRL-Library.git to /private/var/folders/6h/lpcm_n7d1hg47zhs852wxbt40000gn/T/pip-req-build-38817uwr
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/FinRL-Library.git /private/var/folders/6h/lpcm_n7d1hg47zhs852wxbt40000gn/T/pip-req-build-38817uwr


In [2]:
from finrl import config
from finrl import config_tickers
import os
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)


<a id='1.2'></a>
## 2.2. Check if the additional packages needed are present, if not install them. 
* Yahoo Finance API
* pandas
* numpy
* matplotlib
* stockstats
* OpenAI gym
* stable-baselines
* tensorflow
* pyfolio

<a id='1.3'></a>
## 2.3. Import Packages

In [10]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.finrl_meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.finrl_meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.finrl_meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.finrl_meta.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint

import sys
sys.path.append("../FinRL-Library")

import itertools

/Users/yudongtan/miniforge3/lib/python3.9/site-packages/pandas_datareader/compat/__init__.py:11: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  PANDAS_VERSION = LooseVersion(pd.__version__)
/Users/yudongtan/miniforge3/lib/python3.9/site-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(
/Users/yudongtan/miniforge3/lib/python3.9/site-packages/seaborn/rcmod.py:82: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(mpl.__version__) >= "3.0":
/Users/yudongtan/miniforge3/lib/python3.9/site-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)


<a id='1.4'></a>
## 2.4. Create Folders

<a id='2'></a>
# Part 3. Download Data
Yahoo Finance is a website that provides stock data, financial news, financial reports, etc. All the data provided by Yahoo Finance is free.
* FinRL uses a class **YahooDownloader** to fetch data from Yahoo Finance API
* Call Limit: Using the Public API (without authentication), you are limited to 2,000 requests per hour per IP (or up to a total of 48,000 requests a day).




-----
class YahooDownloader:
    Provides methods for retrieving daily stock data from
    Yahoo Finance API

    Attributes
    ----------
        start_date : str
            start date of the data (modified from config.py)
        end_date : str
            end date of the data (modified from config.py)
        ticker_list : list
            a list of stock tickers (modified from config.py)

    Methods
    -------
    fetch_data()
        Fetches data from yahoo API


In [11]:
# from config.py TRAIN_START_DATE is a string
config.TRAIN_START_DATE

'2014-01-01'

In [12]:
# from config.py TRAIN_END_DATE is a string
config.TRAIN_END_DATE

'2020-07-31'

In [13]:
df = YahooDownloader(start_date = '2009-01-01',
                     end_date = '2021-10-31',
                     ticker_list = config_tickers.DOW_30_TICKER).fetch_data()

[*********************100%***********************]  1 of 1 completed


/Users/yudongtan/projects/FinRL/finrl/finrl_meta/preprocessor/yahoodownloader.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_df = data_df.append(temp_df)


[*********************100%***********************]  1 of 1 completed


/Users/yudongtan/projects/FinRL/finrl/finrl_meta/preprocessor/yahoodownloader.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_df = data_df.append(temp_df)


[*********************100%***********************]  1 of 1 completed


...

[*********************100%***********************]  1 of 1 completed


/Users/yudongtan/projects/FinRL/finrl/finrl_meta/preprocessor/yahoodownloader.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_df = data_df.append(temp_df)


[*********************100%***********************]  1 of 1 completed


/Users/yudongtan/projects/FinRL/finrl/finrl_meta/preprocessor/yahoodownloader.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_df = data_df.append(temp_df)


Shape of DataFrame:  (94360, 8)


In [14]:
print(config_tickers.DOW_30_TICKER)

['AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'CRM', 'VZ', 'V', 'WBA', 'WMT', 'DIS', 'DOW']


In [15]:
df.shape

(94360, 8)

In [16]:
df.sort_values(['date','tic'],ignore_index=True).head()

,date,open,high,low,close,volume,tic,day
0,2008-12-31,3.070357,3.133571,3.047857,2.606278,607541200,AAPL,2
1,2008-12-31,57.110001,58.220001,57.060001,43.924454,6287200,AMGN,2
2,2008-12-31,17.969999,18.750000,17.910000,14.908462,9625600,AXP,2
3,2008-12-31,41.590000,43.049999,41.500000,32.005882,5443100,BA,2
4,2008-12-31,43.700001,45.099998,43.700001,30.628832,6277400,CAT,2


# Part 4: Preprocess Data
Data preprocessing is a crucial step for training a high quality machine learning model. We need to check for missing data and do feature engineering in order to convert the data into a model-ready state.
* Add technical indicators. In practical trading, various information needs to be taken into account, for example the historical stock prices, current holding shares, technical indicators, etc. In this article, we demonstrate two trend-following technical indicators: MACD and RSI.
* Add turbulence index. Risk-aversion reflects whether an investor will choose to preserve the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the financial turbulence index that measures extreme asset price fluctuation.

In [17]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = config.INDICATORS,
                    use_vix=True,
                    use_turbulence=True,
                    user_defined_feature = False)

processed = fe.preprocess_data(df)

/Users/yudongtan/projects/FinRL/finrl/finrl_meta/preprocessor/preprocessors.py:154: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = indicator_df.append(
/Users/yudongtan/projects/FinRL/finrl/finrl_meta/preprocessor/preprocessors.py:154: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = indicator_df.append(
/Users/yudongtan/projects/FinRL/finrl/finrl_meta/preprocessor/preprocessors.py:154: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = indicator_df.append(
/Users/yudongtan/projects/FinRL/finrl/finrl_meta/preprocessor/preprocessors.py:154: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = i

/Users/yudongtan/projects/FinRL/finrl/finrl_meta/preprocessor/preprocessors.py:154: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = indicator_df.append(
/Users/yudongtan/projects/FinRL/finrl/finrl_meta/preprocessor/preprocessors.py:154: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = indicator_df.append(
/Users/yudongtan/projects/FinRL/finrl/finrl_meta/preprocessor/preprocessors.py:154: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = indicator_df.append(
/Users/yudongtan/projects/FinRL/finrl/finrl_meta/preprocessor/preprocessors.py:154: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = i

/Users/yudongtan/projects/FinRL/finrl/finrl_meta/preprocessor/preprocessors.py:154: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = indicator_df.append(
/Users/yudongtan/projects/FinRL/finrl/finrl_meta/preprocessor/preprocessors.py:154: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = indicator_df.append(
/Users/yudongtan/projects/FinRL/finrl/finrl_meta/preprocessor/preprocessors.py:154: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = indicator_df.append(
/Users/yudongtan/projects/FinRL/finrl/finrl_meta/preprocessor/preprocessors.py:154: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = i

/Users/yudongtan/projects/FinRL/finrl/finrl_meta/preprocessor/preprocessors.py:154: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = indicator_df.append(
/Users/yudongtan/projects/FinRL/finrl/finrl_meta/preprocessor/preprocessors.py:154: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = indicator_df.append(
/Users/yudongtan/projects/FinRL/finrl/finrl_meta/preprocessor/preprocessors.py:154: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = indicator_df.append(
/Users/yudongtan/projects/FinRL/finrl/finrl_meta/preprocessor/preprocessors.py:154: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = i

/Users/yudongtan/projects/FinRL/finrl/finrl_meta/preprocessor/preprocessors.py:154: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = indicator_df.append(
/Users/yudongtan/projects/FinRL/finrl/finrl_meta/preprocessor/preprocessors.py:154: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = indicator_df.append(
/Users/yudongtan/projects/FinRL/finrl/finrl_meta/preprocessor/preprocessors.py:154: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = indicator_df.append(
/Users/yudongtan/projects/FinRL/finrl/finrl_meta/preprocessor/preprocessors.py:154: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = i

/Users/yudongtan/projects/FinRL/finrl/finrl_meta/preprocessor/preprocessors.py:154: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = indicator_df.append(
/Users/yudongtan/projects/FinRL/finrl/finrl_meta/preprocessor/preprocessors.py:154: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = indicator_df.append(
/Users/yudongtan/projects/FinRL/finrl/finrl_meta/preprocessor/preprocessors.py:154: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = indicator_df.append(
/Users/yudongtan/projects/FinRL/finrl/finrl_meta/preprocessor/preprocessors.py:154: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = i

Successfully added technical indicators
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (3231, 8)
Successfully added vix


/Users/yudongtan/projects/FinRL/finrl/finrl_meta/preprocessor/yahoodownloader.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_df = data_df.append(temp_df)


Successfully added turbulence index


In [18]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)

In [19]:
processed_full.sort_values(['date','tic'],ignore_index=True).head(10)

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2008-12-31,AAPL,3.070357,3.133571,3.047857,2.606278,607541200.0,2.0,0.0,2.921923,2.455528,100.0,66.666667,100.0,2.606278,2.606278,40.0,0.0
1,2008-12-31,AMGN,57.110001,58.220001,57.060001,43.924454,6287200.0,2.0,0.0,2.921923,2.455528,100.0,66.666667,100.0,43.924454,43.924454,40.0,0.0
2,2008-12-31,AXP,17.969999,18.750000,17.910000,14.908462,9625600.0,2.0,0.0,2.921923,2.455528,100.0,66.666667,100.0,14.908462,14.908462,40.0,0.0
3,2008-12-31,BA,41.590000,43.049999,41.500000,32.005882,5443100.0,2.0,0.0,2.921923,2.455528,100.0,66.666667,100.0,32.005882,32.005882,40.0,0.0
4,2008-12-31,CAT,43.700001,45.099998,43.700001,30.628832,6277400.0,2.0,0.0,2.921923,2.455528,100.0,66.666667,100.0,30.628832,30.628832,40.0,0.0
5,2008-12-31,CRM,7.712500,8.130000,7.707500,8.002500,5367600.0,2.0,0.0,2.921923,2.455528,100.0,66.666667,100.0,8.002500,8.002500,40.0,0.0
6,2008-12-31,CSCO,16.180000,16.549999,16.120001,11.787784,37513700.0,2.0,0.0,2.921923,2.455528,100.0,66.666667,100.0,11.787784,11.787784,40.0,0.0
7,2008-12-31,CVX,72.900002,74.629997,72.900002,43.314423,9964300.0,2.0,0.0,2.921923,2.455528,100.0,66.666667,100.0,43.314423,43.314423,40.0,0.0
8,2008-12-31,DIS,22.570000,22.950001,22.520000,19.538345,9012100.0,2.0,0.0,2.921923,2.455528,100.0,66.666667,100.0,19.538345,19.538345,40.0,0.0
9,2008-12-31,GS,82.239998,86.150002,81.120003,68.802803,14894100.0,2.0,0.0,2.921923,2.455528,100.0,66.666667,100.0,68.802803,68.802803,40.0,0.0


<a id='4'></a>
# Part 5. Design Environment
Considering the stochastic and interactive nature of the automated stock trading tasks, a financial task is modeled as a **Markov Decision Process (MDP)** problem. The training process involves observing stock price change, taking an action and reward's calculation to have the agent adjusting its strategy accordingly. By interacting with the environment, the trading agent will derive a trading strategy with the maximized rewards as time proceeds.

Our trading environments, based on OpenAI Gym framework, simulate live stock markets with real market data according to the principle of time-driven simulation.

The action space describes the allowed actions that the agent interacts with the environment. Normally, action a includes three actions: {-1, 0, 1}, where -1, 0, 1 represent selling, holding, and buying one share. Also, an action can be carried upon multiple shares. We use an action space {-k,…,-1, 0, 1, …, k}, where k denotes the number of shares to buy and -k denotes the number of shares to sell. For example, "Buy 10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or -10, respectively. The continuous action space needs to be normalized to [-1, 1], since the policy is defined on a Gaussian distribution, which needs to be normalized and symmetric.

## Training data split: 2009-01-01 to 2020-07-01
## Trade data split: 2020-07-01 to 2021-10-31

In [20]:
train = data_split(processed_full, '2009-01-01','2020-07-01')
trade = data_split(processed_full, '2020-07-01','2021-10-31')
print(len(train))
print(len(trade))

83897
9744


In [21]:
train.tail()

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
2892,2020-06-30,UNH,288.570007,296.450012,287.660004,287.776794,2932900.0,1.0,-0.019478,303.925893,271.251238,52.413044,-25.838425,1.846804,288.020687,281.001443,30.43,12.918791
2892,2020-06-30,V,191.490005,193.750000,190.160004,190.737244,9040100.0,1.0,1.048786,198.750529,185.041397,53.021036,-51.550794,2.013358,191.485038,181.677684,30.43,12.918791
2892,2020-06-30,VZ,54.919998,55.290001,54.360001,50.376740,17414800.0,1.0,-0.437111,53.918427,48.729322,48.097034,-51.018326,8.508886,51.012122,51.464679,30.43,12.918791
2892,2020-06-30,WBA,42.119999,42.580002,41.759998,39.035728,4782100.0,1.0,-0.083987,42.609306,36.487093,48.830175,-14.508170,1.500723,39.135189,38.935129,30.43,12.918791
2892,2020-06-30,WMT,119.220001,120.129997,118.540001,116.121773,6836400.0,1.0,-0.886569,119.473757,113.510459,48.159674,-69.938739,3.847271,117.787628,119.723275,30.43,12.918791


In [22]:
trade.head()

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2020-07-01,AAPL,91.279999,91.839996,90.977501,89.904594,110737200.0,2.0,3.014603,92.699319,80.179953,62.807126,107.491819,29.730532,83.933767,77.717544,28.620001,53.068143
0,2020-07-01,AMGN,235.520004,256.230011,232.580002,240.153946,6575800.0,2.0,3.636393,232.397421,200.212949,61.279630,271.386075,46.806139,214.858661,215.931664,28.620001,53.068143
0,2020-07-01,AXP,95.250000,96.959999,93.639999,92.086380,3301000.0,2.0,-0.389162,110.423945,87.759332,48.504818,-66.328697,3.142448,97.244636,90.695525,28.620001,53.068143
0,2020-07-01,BA,185.880005,190.610001,180.039993,180.320007,49036700.0,2.0,5.443193,220.721139,160.932863,50.925771,24.220608,15.932920,176.472335,155.614168,28.620001,53.068143
0,2020-07-01,CAT,129.380005,129.399994,125.879997,120.651634,2807800.0,2.0,1.272630,130.624272,113.353134,52.865421,35.597292,14.457404,119.412837,113.646674,28.620001,53.068143


In [23]:
config.INDICATORS

['macd',
 'boll_ub',
 'boll_lb',
 'rsi_30',
 'cci_30',
 'dx_30',
 'close_30_sma',
 'close_60_sma']

In [24]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(config.INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")


Stock Dimension: 29, State Space: 291


In [25]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": config.INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}


e_train_gym = StockTradingEnv(df = train, **env_kwargs)

## Environment for Training



In [26]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


<a id='5'></a>
# Part 6: Implement DRL Algorithms
* The implementation of the DRL algorithms are based on **OpenAI Baselines** and **Stable Baselines**. Stable Baselines is a fork of OpenAI Baselines, with a major structural refactoring, and code cleanups.
* FinRL library includes fine-tuned standard DRL algorithms, such as DQN, DDPG,
Multi-Agent DDPG, PPO, SAC, A2C and TD3. We also allow users to
design their own DRL algorithms by adapting these DRL algorithms.

In [27]:
agent = DRLAgent(env = env_train)

### Model Training: 5 models, A2C DDPG, PPO, TD3, SAC


### Model 1: A2C


In [28]:
agent = DRLAgent(env = env_train)
model_a2c = agent.get_model("a2c")

{'n_steps': 5, 'ent_coef': 0.01, 'learning_rate': 0.0007}
Using cpu device


In [29]:
trained_a2c = agent.train_model(model=model_a2c, 
                             tb_log_name='a2c',
                             total_timesteps=50000)

-------------------------------------
| time/                 |           |
|    fps                | 244       |
|    iterations         | 100       |
|    time_elapsed       | 2         |
|    total_timesteps    | 500       |
| train/                |           |
|    entropy_loss       | -41.3     |
|    explained_variance | -0.274    |
|    learning_rate      | 0.0007    |
|    n_updates          | 99        |
|    policy_loss        | -21.4     |
|    reward             | 0.2950889 |
|    std                | 1         |
|    value_loss         | 0.634     |
-------------------------------------
-------------------------------------
| time/                 |           |
|    fps                | 245       |
|    iterations         | 200       |
|    time_elapsed       | 4         |
|    total_timesteps    | 1000      |
| train/                |           |
|    entropy_loss       | -41.3     |
|    explained_variance | -0.0277   |
|    learning_rate      | 0.0007    |
|    n_updat

-------------------------------------
| time/                 |           |
|    fps                | 249       |
|    iterations         | 1500      |
|    time_elapsed       | 30        |
|    total_timesteps    | 7500      |
| train/                |           |
|    entropy_loss       | -41.7     |
|    explained_variance | 0         |
|    learning_rate      | 0.0007    |
|    n_updates          | 1499      |
|    policy_loss        | 191       |
|    reward             | 3.1103804 |
|    std                | 1.02      |
|    value_loss         | 28.5      |
-------------------------------------
--------------------------------------
| time/                 |            |
|    fps                | 249        |
|    iterations         | 1600       |
|    time_elapsed       | 32         |
|    total_timesteps    | 8000       |
| train/                |            |
|    entropy_loss       | -41.7      |
|    explained_variance | 0          |
|    learning_rate      | 0.0007     |
| 

-------------------------------------
| time/                 |           |
|    fps                | 249       |
|    iterations         | 2900      |
|    time_elapsed       | 58        |
|    total_timesteps    | 14500     |
| train/                |           |
|    entropy_loss       | -42       |
|    explained_variance | 0         |
|    learning_rate      | 0.0007    |
|    n_updates          | 2899      |
|    policy_loss        | -113      |
|    reward             | 1.7608367 |
|    std                | 1.03      |
|    value_loss         | 6.62      |
-------------------------------------
--------------------------------------
| time/                 |            |
|    fps                | 249        |
|    iterations         | 3000       |
|    time_elapsed       | 60         |
|    total_timesteps    | 15000      |
| train/                |            |
|    entropy_loss       | -42.1      |
|    explained_variance | 0          |
|    learning_rate      | 0.0007     |
| 

------------------------------------
| time/                 |          |
|    fps                | 250      |
|    iterations         | 4300     |
|    time_elapsed       | 85       |
|    total_timesteps    | 21500    |
| train/                |          |
|    entropy_loss       | -42.3    |
|    explained_variance | 0        |
|    learning_rate      | 0.0007   |
|    n_updates          | 4299     |
|    policy_loss        | -9.05    |
|    reward             | 4.037478 |
|    std                | 1.04     |
|    value_loss         | 1.38     |
------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 250      |
|    iterations         | 4400     |
|    time_elapsed       | 87       |
|    total_timesteps    | 22000    |
| train/                |          |
|    entropy_loss       | -42.4    |
|    explained_variance | 0        |
|    learning_rate      | 0.0007   |
|    n_updates          | 4399     |
|

-------------------------------------
| time/                 |           |
|    fps                | 250       |
|    iterations         | 5700      |
|    time_elapsed       | 113       |
|    total_timesteps    | 28500     |
| train/                |           |
|    entropy_loss       | -42.8     |
|    explained_variance | 0         |
|    learning_rate      | 0.0007    |
|    n_updates          | 5699      |
|    policy_loss        | -588      |
|    reward             | -7.023079 |
|    std                | 1.06      |
|    value_loss         | 225       |
-------------------------------------
--------------------------------------
| time/                 |            |
|    fps                | 250        |
|    iterations         | 5800       |
|    time_elapsed       | 115        |
|    total_timesteps    | 29000      |
| train/                |            |
|    entropy_loss       | -42.8      |
|    explained_variance | -170       |
|    learning_rate      | 0.0007     |
| 

-------------------------------------
| time/                 |           |
|    fps                | 250       |
|    iterations         | 7100      |
|    time_elapsed       | 141       |
|    total_timesteps    | 35500     |
| train/                |           |
|    entropy_loss       | -43.3     |
|    explained_variance | 0         |
|    learning_rate      | 0.0007    |
|    n_updates          | 7099      |
|    policy_loss        | 2.83      |
|    reward             | 0.6510213 |
|    std                | 1.08      |
|    value_loss         | 0.229     |
-------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 250      |
|    iterations         | 7200     |
|    time_elapsed       | 143      |
|    total_timesteps    | 36000    |
| train/                |          |
|    entropy_loss       | -43.3    |
|    explained_variance | 0        |
|    learning_rate      | 0.0007   |
|    n_updates        

---------------------------------------
| time/                 |             |
|    fps                | 250         |
|    iterations         | 8500        |
|    time_elapsed       | 169         |
|    total_timesteps    | 42500       |
| train/                |             |
|    entropy_loss       | -43.7       |
|    explained_variance | 0           |
|    learning_rate      | 0.0007      |
|    n_updates          | 8499        |
|    policy_loss        | -122        |
|    reward             | -0.63955355 |
|    std                | 1.1         |
|    value_loss         | 8.84        |
---------------------------------------
--------------------------------------
| time/                 |            |
|    fps                | 61         |
|    iterations         | 8600       |
|    time_elapsed       | 704        |
|    total_timesteps    | 43000      |
| train/                |            |
|    entropy_loss       | -43.8      |
|    explained_variance | 0          |
|    lear

---------------------------------------
| time/                 |             |
|    fps                | 67          |
|    iterations         | 9900        |
|    time_elapsed       | 730         |
|    total_timesteps    | 49500       |
| train/                |             |
|    entropy_loss       | -43.9       |
|    explained_variance | -4.77e-07   |
|    learning_rate      | 0.0007      |
|    n_updates          | 9899        |
|    policy_loss        | -4.21       |
|    reward             | -0.05840838 |
|    std                | 1.1         |
|    value_loss         | 0.0601      |
---------------------------------------
-------------------------------------
| time/                 |           |
|    fps                | 68        |
|    iterations         | 10000     |
|    time_elapsed       | 732       |
|    total_timesteps    | 50000     |
| train/                |           |
|    entropy_loss       | -43.9     |
|    explained_variance | -1.19e-07 |
|    learning_rate

### Model 2: DDPG

In [30]:
agent = DRLAgent(env = env_train)
model_ddpg = agent.get_model("ddpg")

{'batch_size': 128, 'buffer_size': 50000, 'learning_rate': 0.001}
Using cpu device


In [31]:
trained_ddpg = agent.train_model(model=model_ddpg, 
                             tb_log_name='ddpg',
                             total_timesteps=50000)

day: 2892, episode: 20
begin_total_asset: 1000000.00
end_total_asset: 5275602.43
total_reward: 4275602.43
total_cost: 5627.13
total_trades: 36256
Sharpe: 0.946
----------------------------------
| time/              |           |
|    episodes        | 4         |
|    fps             | 140       |
|    time_elapsed    | 82        |
|    total_timesteps | 11572     |
| train/             |           |
|    actor_loss      | -26.8     |
|    critic_loss     | 135       |
|    learning_rate   | 0.001     |
|    n_updates       | 8679      |
|    reward          | 10.533023 |
----------------------------------
----------------------------------
| time/              |           |
|    episodes        | 8         |
|    fps             | 129       |
|    time_elapsed    | 178       |
|    total_timesteps | 23144     |
| train/             |           |
|    actor_loss      | -18.3     |
|    critic_loss     | 14.7      |
|    learning_rate   | 0.001     |
|    n_updates       | 20251     |


### Model 3: PPO

In [ ]:
agent = DRLAgent(env = env_train)
PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.01,
    "learning_rate": 0.00025,
    "batch_size": 128,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

In [ ]:
trained_ppo = agent.train_model(model=model_ppo, 
                             tb_log_name='ppo',
                             total_timesteps=50000)

### Model 4: TD3

In [ ]:
agent = DRLAgent(env = env_train)
TD3_PARAMS = {"batch_size": 100, 
              "buffer_size": 1000000, 
              "learning_rate": 0.001}

model_td3 = agent.get_model("td3",model_kwargs = TD3_PARAMS)

In [ ]:
trained_td3 = agent.train_model(model=model_td3, 
                             tb_log_name='td3',
                             total_timesteps=30000)

### Model 5: SAC

In [32]:
agent = DRLAgent(env = env_train)
SAC_PARAMS = {
    "batch_size": 128,
    "buffer_size": 1000000,
    "learning_rate": 0.0001,
    "learning_starts": 100,
    "ent_coef": "auto_0.1",
}

model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)

{'batch_size': 128, 'buffer_size': 1000000, 'learning_rate': 0.0001, 'learning_starts': 100, 'ent_coef': 'auto_0.1'}
Using cpu device


In [33]:
trained_sac = agent.train_model(model=model_sac, 
                             tb_log_name='sac',
                             total_timesteps=60000)

day: 2892, episode: 40
begin_total_asset: 1000000.00
end_total_asset: 3750425.08
total_reward: 2750425.08
total_cost: 129986.08
total_trades: 47122
Sharpe: 0.729
----------------------------------
| time/              |           |
|    episodes        | 4         |
|    fps             | 101       |
|    time_elapsed    | 114       |
|    total_timesteps | 11572     |
| train/             |           |
|    actor_loss      | 424       |
|    critic_loss     | 63.4      |
|    ent_coef        | 0.152     |
|    ent_coef_loss   | -91.1     |
|    learning_rate   | 0.0001    |
|    n_updates       | 11471     |
|    reward          | 1.8715892 |
----------------------------------
---------------------------------
| time/              |          |
|    episodes        | 8        |
|    fps             | 101      |
|    time_elapsed    | 227      |
|    total_timesteps | 23144    |
| train/             |          |
|    actor_loss      | 111      |
|    critic_loss     | 83.1     |
|    en

## Trading
Assume that we have $1,000,000 initial capital at 2020-07-01. We use the DDPG model to trade Dow jones 30 stocks.

### Set turbulence threshold
Set the turbulence threshold to be greater than the maximum of insample turbulence data, if current turbulence index is greater than the threshold, then we assume that the current market is volatile

In [34]:
data_risk_indicator = processed_full[(processed_full.date<'2020-07-01') & (processed_full.date>='2009-01-01')]
insample_risk_indicator = data_risk_indicator.drop_duplicates(subset=['date'])

In [35]:
insample_risk_indicator.vix.describe()

count    2893.000000
mean       18.824245
std         8.489311
min         9.140000
25%        13.330000
50%        16.139999
75%        21.309999
max        82.690002
Name: vix, dtype: float64

In [36]:
insample_risk_indicator.vix.quantile(0.996)

57.40400183105453

In [37]:
insample_risk_indicator.turbulence.describe()

count    2893.000000
mean       34.574232
std        43.787144
min         0.000000
25%        14.966415
50%        24.124362
75%        39.161957
max       652.503824
Name: turbulence, dtype: float64

In [38]:
insample_risk_indicator.turbulence.quantile(0.996)

276.4516803931362

### Trade

DRL model needs to update periodically in order to take full advantage of the data, ideally we need to retrain our model yearly, quarterly, or monthly. We also need to tune the parameters along the way, in this notebook I only use the in-sample data from 2009-01 to 2020-07 to tune the parameters once, so there is some alpha decay here as the length of trade date extends. 

Numerous hyperparameters – e.g. the learning rate, the total number of samples to train on – influence the learning process and are usually determined by testing some variations.

In [39]:
#trade = data_split(processed_full, '2020-07-01','2021-10-31')
e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = 70,risk_indicator_col='vix', **env_kwargs)
# env_trade, obs_trade = e_trade_gym.get_sb_env()

In [40]:
trade.head()

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2020-07-01,AAPL,91.279999,91.839996,90.977501,89.904594,110737200.0,2.0,3.014603,92.699319,80.179953,62.807126,107.491819,29.730532,83.933767,77.717544,28.620001,53.068143
0,2020-07-01,AMGN,235.520004,256.230011,232.580002,240.153946,6575800.0,2.0,3.636393,232.397421,200.212949,61.279630,271.386075,46.806139,214.858661,215.931664,28.620001,53.068143
0,2020-07-01,AXP,95.250000,96.959999,93.639999,92.086380,3301000.0,2.0,-0.389162,110.423945,87.759332,48.504818,-66.328697,3.142448,97.244636,90.695525,28.620001,53.068143
0,2020-07-01,BA,185.880005,190.610001,180.039993,180.320007,49036700.0,2.0,5.443193,220.721139,160.932863,50.925771,24.220608,15.932920,176.472335,155.614168,28.620001,53.068143
0,2020-07-01,CAT,129.380005,129.399994,125.879997,120.651634,2807800.0,2.0,1.272630,130.624272,113.353134,52.865421,35.597292,14.457404,119.412837,113.646674,28.620001,53.068143


In [41]:
df_account_value, df_actions = DRLAgent.DRL_prediction(
    model=trained_sac, 
    environment = e_trade_gym)

hit end!


In [42]:
df_account_value.shape

(336, 2)

In [43]:
df_account_value.tail()

,date,account_value
331,2021-10-22,1.450417e+06
332,2021-10-25,1.456955e+06
333,2021-10-26,1.457912e+06
334,2021-10-27,1.438427e+06
335,2021-10-28,1.448063e+06


In [44]:
df_actions.head()

,AAPL,AMGN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,GS,...,MRK,MSFT,NKE,PG,TRV,UNH,V,VZ,WBA,WMT
date,,,,,,,,,,,,,,,,,,,,,
2020-07-01,96,0,0,92,91,96,93,0,79,97,...,0,0,0,97,0,0,97,89,0,0
2020-07-02,96,0,0,92,91,96,93,0,79,97,...,0,0,0,97,0,0,97,89,0,0
2020-07-06,96,0,0,92,91,96,93,0,79,97,...,0,0,0,97,0,0,97,89,0,0
2020-07-07,96,0,0,92,91,96,93,0,79,97,...,0,0,0,97,0,0,97,89,0,0
2020-07-08,96,0,0,92,91,96,93,0,79,97,...,0,0,0,97,0,0,97,89,0,0


<a id='6'></a>
# Part 7: Backtest Our Strategy
Backtesting plays a key role in evaluating the performance of a trading strategy. Automated backtesting tool is preferred because it reduces the human error. We usually use the Quantopian pyfolio package to backtest our trading strategies. It is easy to use and consists of various individual plots that provide a comprehensive image of the performance of a trading strategy.

<a id='6.1'></a>
## 7.1 BackTestStats
pass in df_account_value, this information is stored in env class


In [45]:
print("==============Get Backtest Results===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+config.RESULTS_DIR+"/perf_stats_all_"+now+'.csv')

==============Get Backtest Results===========
Annual return          0.320051
Cumulative returns     0.448063
Annual volatility      0.155290
Sharpe ratio           1.871866
Calmar ratio           3.111252
Stability              0.916583
Max drawdown          -0.102869
Omega ratio            1.377465
Sortino ratio          2.803189
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.136893
Daily value at risk   -0.018411
dtype: float64


/Users/yudongtan/miniforge3/lib/python3.9/site-packages/pyfolio/timeseries.py:724: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  stats = pd.Series()


In [46]:
#baseline stats
print("==============Get Baseline Stats===========")
baseline_df = get_baseline(
        ticker="^DJI", 
        start = df_account_value.loc[0,'date'],
        end = df_account_value.loc[len(df_account_value)-1,'date'])

stats = backtest_stats(baseline_df, value_col_name = 'close')


==============Get Baseline Stats===========
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (336, 8)
Annual return          0.269722
Cumulative returns     0.374922
Annual volatility      0.139083
Sharpe ratio           1.792302
Calmar ratio           3.020136
Stability              0.919220
Max drawdown          -0.089308
Omega ratio            1.347571
Sortino ratio          2.655481
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.052781
Daily value at risk   -0.016534
dtype: float64


/Users/yudongtan/projects/FinRL/finrl/finrl_meta/preprocessor/yahoodownloader.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_df = data_df.append(temp_df)
/Users/yudongtan/miniforge3/lib/python3.9/site-packages/pyfolio/timeseries.py:724: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  stats = pd.Series()


In [47]:
df_account_value.loc[0,'date']

'2020-07-01'

In [48]:
df_account_value.loc[len(df_account_value)-1,'date']

'2021-10-28'

<a id='6.2'></a>
## 7.2 BackTestPlot

In [62]:
print("==============Compare to DJIA===========")
%matplotlib inline
%reload_ext autoreload
%autoreload 2
import pyfolio
import importlib
importlib.reload(pyfolio)
import finrl
importlib.reload(finrl)
importlib.reload(finrl.plot)
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
# S&P 500: ^GSPC
# Dow Jones Index: ^DJI
# NASDAQ 100: ^NDX
backtest_plot(df_account_value, 
             baseline_ticker = '^DJI', 
             baseline_start = df_account_value.loc[0,'date'],
             baseline_end = df_account_value.loc[len(df_account_value)-1,'date'])

==============Compare to DJIA===========
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (336, 8)


/Users/yudongtan/projects/FinRL/finrl/finrl_meta/preprocessor/yahoodownloader.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_df = data_df.append(temp_df)
/Users/yudongtan/miniforge3/lib/python3.9/site-packages/seaborn/rcmod.py:400: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(mpl.__version__) >= "3.0":
/Users/yudongtan/miniforge3/lib/python3.9/site-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/Users/yudongtan/miniforge3/lib/python3.9/site-packages/pyfolio/timeseries.py:724: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  stats = pd.Series()


Start date,2020-07-01
End date,2021-10-28
Total months,16
,Backtest
Annual return,32.0%
Cumulative returns,44.8%
Annual volatility,15.5%
Sharpe ratio,1.87
Calmar ratio,3.11
Stability,0.92
Max drawdown,-10.3%


AttributeError: 'numpy.int64' object has no attribute 'to_pydatetime'